# Bleaching samples

## update sample type 
- everything is healthy or diseased right now, 
-> change to bleached based sampling notes
-> then crosscheck with colonydata?

In [2]:
import numpy as np
import pandas as pd
import os
import re

In [15]:
# upload sample and colony data 
sample=pd.read_csv('../sctld/SCTLD_samples/Sample_Data/CBC_samples.csv')
colony=pd.read_csv('../sctld/SCTLD_samples/Sample_Data/CBC_ColonyData.csv')

In [16]:
sample.head()

,Month_year,Country,Location,CollectionDate,Transect,TransectNum,OldTagNum,NewTagNum,Species,Time_sampled,Time_processed,Sample_type,SampleNum,Health_status,Sampling_notes,Tubelabel_species,Sample_physical_location,Extraction_physical_location,Date_sequenced,Notes
0,122022,BEL,CBC,12/5/22,CURLEW,4,NaN,77,OFAV,NaN,NaN,Core_frozen,18,Healthy,NaN,122022_BEL_CBC_T4_18_OFAV,NaN,NaN,NaN,NaN
1,92023,BEL,CBC,9/27/23,CBC30N,1,NaN,1,SSID,NaN,NaN,Core_RNAlater,185,Diseased_Margin,only margin sample available,092023_BEL_CBC_T1_185_SSID,UML_NARWHAL_R1_B10,NaN,NaN,NaN
2,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,2,PAST,NaN,NaN,Core_RNAlater,171,Healthy,CLP 90%,092023_BEL_CBC_T1_171_PAST,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN
3,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,3,SSID,NaN,NaN,Core_RNAlater,173,Healthy,CLP 80%; DC 20%,092023_BEL_CBC_T1_173_SSID,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN
4,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,12,PSTR,NaN,NaN,Core_RNAlater,177,Healthy,No CL,092023_BEL_CBC_T1_177_PSTR,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN


In [17]:
colony.head()

,Unnamed: 0,Date_InitialTag,Transect,TransectNum,OldTagNum,NewTagNum,Species,Meter,Meters_90,Direction,...,082024_Percentage,Notes_082024,122024_Condition,122024_Percentage,Notes_122024,062025_Condition,062025_Percentage,Notes_062025,immune_y/n,checked_colonies
0,1.0,6/21/19,CBC30N,1,349,1,SSID,1.5,0.5,right,...,NaN,NaN,Not_visited,NaN,Diseased,Diseased,NaN,NaN,n,x
1,2.0,6/21/19,CBC30N,1,334,2,PAST,1.4,0.1,left,...,NaN,sample lower colony (original colony); inc old...,NaN,NaN,NaN,Healthy,NaN,NaN,y,x
2,3.0,6/21/19,CBC30N,1,346,3,SSID,6.3,0.8,right,...,NaN,90% CLP; some discoloration,DC,10%,30% mortality,Diseased,NaN,NaN,y,x
3,4.0,10/15/19,CBC30N,1,407,4,PSTR,6.5,0.5,right,...,NaN,NaN,Not_visited,NaN,Dead,NaN,NaN,NaN,n,x
4,5.0,6/24/19,CBC30N,1,394,5,SSID,4.3,1.2,left,...,NaN,NaN,DC,10%,80% mortality,Diseased,NaN,NaN,n,x


In [18]:
# search for sampling notes with clp or clb 

#if col contains clb or clp 
# replace status with paled or bleached tissue
sample.loc[
    sample['Sampling_notes'].str.contains('CLP', case=False, na=False),
    'Health_status'] = 'Paled_Tissue'

# CLB → Bleached_Tissue
sample.loc[
    sample['Sampling_notes'].str.contains('CLB', case=False, na=False),
    'Health_status'] = 'Bleached_Tissue'

In [26]:
clb_sample = sample[
    (sample['Health_status'] == 'Bleached_Tissue') |
    (sample['Health_status'] == 'Paled_Tissue')
]

In [27]:
clb_sample

,Month_year,Country,Location,CollectionDate,Transect,TransectNum,OldTagNum,NewTagNum,Species,Time_sampled,Time_processed,Sample_type,SampleNum,Health_status,Sampling_notes,Tubelabel_species,Sample_physical_location,Extraction_physical_location,Date_sequenced,Notes
2,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,2,PAST,NaN,NaN,Core_RNAlater,171,Paled_Tissue,CLP 90%,092023_BEL_CBC_T1_171_PAST,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN
3,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,3,SSID,NaN,NaN,Core_RNAlater,173,Paled_Tissue,CLP 80%; DC 20%,092023_BEL_CBC_T1_173_SSID,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN
12,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,24,MCAV,NaN,NaN,Core_RNAlater,174,Paled_Tissue,CLP 10%,092023_BEL_CBC_T1_174_MCAV,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN
13,92023,BEL,CBC,9/27/23,CBC30N,1,NaN,25,OANN,NaN,NaN,Core_RNAlater,178,Paled_Tissue,CLP 15%,092023_BEL_CBC_T1_178_OANN,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN
14,92023,BEL,CBC,9/27/23,CBC30N,1,NaN,16,SSID,NaN,NaN,Core_RNAlater,194,Bleached_Tissue,CLB 90%,092023_BEL_CBC_T1_194_SSID,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B11,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2268,62025,BEL,CBC,5/30/25,CBC30N,1,NaN,22,OANN,9:39,10:49,Core_RNAlater,1619,Paled_Tissue,"Colony 60% clp, Healthy sample",062025_BEL_CBC_T1_1619_OANN,UML_Narwhal_R3_B43,NaN,NaN,NaN
2269,62025,BEL,CBC,5/30/25,CBC30N,1,NaN,25,OANN,9:50,10:51,Core_frozen,1820,Bleached_Tissue,"Colony 10% CLB, Healthy sample",NaN,NaN,NaN,NaN,NaN
2270,62025,BEL,CBC,5/30/25,CBC30N,1,NaN,25,OANN,9:50,10:51,Core_RNAlater,1620,Bleached_Tissue,"Colony 10% CLB, Healthy sample",062025_BEL_CBC_T1_1620_OANN,UML_Narwhal_R3_B43,NaN,NaN,NaN
2491,62025,BEL,CBC,6/2/25,Hangman,6,NaN,26,OANN,10:58,11:32,Core_frozen,1980,Paled_Tissue,10% CLP,NaN,NaN,NaN,NaN,NaN


In [ ]:
# other option is to match the colony data with sample data and based on the percentage, fix the tissue vs margin?